In [ ]:
# 📦 Gerekli kütüphanelerin import edilmesi
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
import gradio as gr

from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_curve, auc

nltk.download('stopwords')
turkce_stopwords = set(stopwords.words('turkish'))


In [ ]:
# 🧹 Metin temizleme fonksiyonu
def temizle(yorum):
    yorum = str(yorum).lower()
    yorum = re.sub(r"http\\S+", "", yorum)
    yorum = re.sub(r"[^a-zçğıöşü0-9\\s]", "", yorum)
    yorum = re.sub(r"\\s+", " ", yorum).strip()
    yorum = " ".join([w for w in yorum.split() if w not in turkce_stopwords])
    return yorum


In [ ]:
# 🗃️ Veri setini yükleme ve temizleme işlemleri
df = pd.read_csv("eticaret-urun-yorumlari.csv", sep=";")
df.columns = ["Metin", "Durum"]
df = df[df["Durum"].isin([0, 1])]
df["temiz_yorum"] = df["Metin"].apply(temizle)
df = df[df["temiz_yorum"].str.len() > 5]
df = df.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# ☁️ Kelime bulutu ile genel görünüm
wc = WordCloud(width=1000, height=500, background_color='white', colormap='viridis', max_words=200).generate(" ".join(df["temiz_yorum"]))
plt.figure(figsize=(14, 6))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off")
plt.title("Kelime Bulutu – Temizlenmiş Yorumlar", fontsize=16, color="darkblue")
plt.show()


In [ ]:
# 📊 Eğitim ve test verisinin ayrılması (60% eğitim, 40% test)
X_train, X_test, y_train, y_test = train_test_split(df["temiz_yorum"], df["Durum"], test_size=0.4, random_state=42)


In [ ]:
# 🔄 Geliştirilmiş TF-IDF Vektörleştirme (Overfitting azaltmak için)
vectorizer = TfidfVectorizer(ngram_range=(1, 1), min_df=1, max_df=0.95)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
# 🧠 Logistic Regression modeli (class_weight ile dengelenmiş)
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(class_weight='balanced', max_iter=1000)
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)


In [ ]:
# 🧠 MultinomialNB modeli için sadeleştirilmiş TF-IDF ve eğitim
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Daha sade TF-IDF
vectorizer_nb = TfidfVectorizer(
    ngram_range=(1, 1),   # sadece tek kelimeler
    min_df=3,             # en az 3 belgede geçen kelimeler
    max_df=0.95,          # %95'ten fazla geçenleri çıkar
    max_features=5000     # sadece en anlamlı 5000 kelime
)

X_train_vec_nb = vectorizer_nb.fit_transform(X_train)
X_test_vec_nb = vectorizer_nb.transform(X_test)

# Modeli eğit
model_nb = MultinomialNB(alpha=0.5)  # smoothing değeri (daha dengeli sonuçlar verir)
model_nb.fit(X_train_vec_nb, y_train)


In [ ]:
# 🔍 En Etkili Kelimeleri Görselleştir – MultinomialNB (Log odds farkı)
import numpy as np
import matplotlib.pyplot as plt

log_probs = model_nb.feature_log_prob_
importance = log_probs[1] - log_probs[0]  # Olumlu - Olumsuz

# En olumlu ve olumsuz kelimeleri sırala
top_positive_idx = np.argsort(importance)[-15:]
top_negative_idx = np.argsort(importance)[:15]

feature_names = np.array(vectorizer_nb.get_feature_names_out())
top_pos_words = feature_names[top_positive_idx]
top_pos_values = importance[top_positive_idx]
top_neg_words = feature_names[top_negative_idx]
top_neg_values = importance[top_negative_idx]

# Grafik çiz
plt.figure(figsize=(12, 6))
plt.barh(top_pos_words, top_pos_values, color='green', label="Olumlu Etki")
plt.barh(top_neg_words, top_neg_values, color='red', label="Olumsuz Etki")
plt.title("MultinomialNB: Tahmine En Çok Etki Eden Kelimeler", fontsize=14, color="darkblue")
plt.xlabel("Etki Skoru (log odds farkı)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 📉 Confusion Matrix – Isı haritası şeklinde görselleştirme
cm = confusion_matrix(y_test, y_pred)
etiketler = ["Olumsuz", "Olumlu"]

plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt="d", cmap="YlGnBu", xticklabels=etiketler, yticklabels=etiketler, cbar=False)
plt.title("🎯 Confusion Matrix – MultinomialNB", fontsize=14, color="darkgreen")
plt.xlabel("Tahmin Edilen")
plt.ylabel("Gerçek Değer")
plt.tight_layout()
plt.show()


In [ ]:
# 🚀 ROC Eğrisi (Receiver Operating Characteristic)
y_probs = model.predict_proba(X_test_vec)[:, 1]  # Olumlu sınıf olasılığı
fpr, tpr, _ = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f"ROC eğrisi (AUC = {roc_auc:.2f})")
plt.plot([0, 1], [0, 1], color='navy', linestyle='--', lw=1)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – MultinomialNB")
plt.legend(loc="lower right")
plt.grid(True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# 🔍 Tahmini etkileyen en önemli kelimeleri göster
log_probs = model.feature_log_prob_
importance = log_probs[1] - log_probs[0]

top_positive_idx = np.argsort(importance)[-15:]
top_negative_idx = np.argsort(importance)[:15]

feature_names = np.array(vectorizer.get_feature_names_out())
top_pos_words = feature_names[top_positive_idx]
top_pos_values = importance[top_positive_idx]
top_neg_words = feature_names[top_negative_idx]
top_neg_values = importance[top_negative_idx]

plt.figure(figsize=(12, 6))
plt.barh(top_pos_words, top_pos_values, color='green', label="Olumlu Etki")
plt.barh(top_neg_words, top_neg_values, color='red', label="Olumsuz Etki")
plt.title("MultinomialNB: Tahmine En Çok Etki Eden Kelimeler")
plt.xlabel("Etki Skoru (log odds farkı)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
# 🧪 Farklı modellerin doğruluk karşılaştırma grafiği
data = {
    "Model": ["Logistic Regression", "Multinomial NB", "Linear SVC"],
    "Eğitim Doğruluğu": [0.9744, 0.9740, 0.9948],
    "Test Doğruluğu": [0.9497, 0.9556, 0.9552]
}

df_karsilastirma = pd.DataFrame(data)

plt.figure(figsize=(8, 5))
bar_width = 0.3
index = np.arange(len(df_karsilastirma))

plt.bar(index, df_karsilastirma["Eğitim Doğruluğu"], bar_width, label="Eğitim", color="#4CAF50")
plt.bar(index + bar_width, df_karsilastirma["Test Doğruluğu"], bar_width, label="Test", color="#2196F3")

plt.xlabel("Model", fontsize=12)
plt.ylabel("Doğruluk", fontsize=12)
plt.title("📊 Model Doğruluk Karşılaştırması", fontsize=14, weight="bold")
plt.xticks(index + bar_width / 2, df_karsilastirma["Model"], rotation=15)
plt.ylim(0.90, 1.00)
plt.legend()
plt.grid(True, linestyle="--", alpha=0.4)
plt.tight_layout()
plt.show()


In [ ]:
# 📲 Gradio Sekmeli Arayüz: Tahmin + Performans + Model Karşılaştırması + Etkili Kelimeler
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# 🎯 Model Karşılaştırma Grafiği Fonksiyonu
def model_karsilastirma_grafigi():
    data = {
        "Model": ["Logistic Regression", "Multinomial NB", "Linear SVC"],
        "Eğitim Doğruluğu": [0.9744, 0.9740, 0.9948],
        "Test Doğruluğu": [0.9497, 0.9556, 0.9552]
    }
    df = pd.DataFrame(data)
    plt.figure(figsize=(8, 5))
    bar_width = 0.3
    index = np.arange(len(df))

    plt.bar(index, df["Eğitim Doğruluğu"], bar_width, label="Eğitim", color="#4CAF50")
    plt.bar(index + bar_width, df["Test Doğruluğu"], bar_width, label="Test", color="#2196F3")

    plt.xlabel("Model", fontsize=12)
    plt.ylabel("Doğruluk", fontsize=12)
    plt.title("Model Doğruluk Karşılaştırması", fontsize=14, weight="bold")
    plt.xticks(index + bar_width / 2, df["Model"], rotation=15)
    plt.ylim(0.90, 1.00)
    plt.legend()
    plt.grid(True, linestyle="--", alpha=0.4)
    plt.tight_layout()

    path = "model_karsilastirma.png"
    plt.savefig(path)
    plt.close()
    return path

# 📌 En Etkili Kelimeler Görselleştirme
def etkili_kelimeler_grafigi():
    log_probs = model_nb.feature_log_prob_
    importance = log_probs[1] - log_probs[0]

    top_positive_idx = np.argsort(importance)[-15:]
    top_negative_idx = np.argsort(importance)[:15]

    feature_names = np.array(vectorizer_nb.get_feature_names_out())
    top_pos_words = feature_names[top_positive_idx]
    top_pos_values = importance[top_positive_idx]
    top_neg_words = feature_names[top_negative_idx]
    top_neg_values = importance[top_negative_idx]

    plt.figure(figsize=(12, 6))
    plt.barh(top_pos_words, top_pos_values, color='green', label="Olumlu Etki")
    plt.barh(top_neg_words, top_neg_values, color='red', label="Olumsuz Etki")
    plt.title("MultinomialNB: Tahmine En Çok Etki Eden Kelimeler")
    plt.xlabel("Etki Skoru (log odds farkı)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    path = "etkili_kelimeler.png"
    plt.savefig(path)
    plt.close()
    return path

# 🎛️ Gradio Arayüzü
with gr.Blocks(theme=gr.themes.Soft()) as demo:

    # 🛍️ Tahmin Sekmesi (kullanıcı kaldırmak istemişti)
    with gr.Tab("📊 Performans"):
        gr.Markdown("### Model Performansı\nEğitim/Test doğruluğu ve AUC skoru")

        def performans_ozeti():
            train_acc = model.score(X_train_vec, y_train)
            test_acc = model.score(X_test_vec, y_test)
            y_probs = model.predict_proba(X_test_vec)[:, 1]
            fpr, tpr, _ = roc_curve(y_test, y_probs)
            roc_auc_val = auc(fpr, tpr)

            return f"""
            <div style='font-size:16px'>
                <b>📘 Eğitim Doğruluğu:</b> %{train_acc*100:.2f}<br>
                <b>📗 Test Doğruluğu:</b> %{test_acc*100:.2f}<br>
                <b>📈 AUC Skoru:</b> {roc_auc_val:.3f}<br><br>
                <i>Model: Multinomial Naive Bayes</i>
            </div>
            """

        performans_output = gr.HTML()
        gr.Button("Performansı Göster").click(fn=performans_ozeti, outputs=performans_output)

    # 📉 Model Karşılaştırması Sekmesi
    with gr.Tab("📉 Model Karşılaştırması"):
        gr.Markdown("### Modellerin Eğitim ve Test Başarıları (Görsel)")
        grafik_output = gr.Image(label="Model Karşılaştırma Grafiği")
        gr.Button("Grafiği Göster").click(fn=model_karsilastirma_grafigi, outputs=grafik_output)

    # 🔍 En Etkili Kelimeler Sekmesi
    with gr.Tab("🧠 En Etkili Kelimeler"):
        gr.Markdown("### MultinomialNB ile Tahmine En Çok Etki Eden Kelimeler")
        kelime_output = gr.Image(label="Etki Skoru ile Kelimeler")
        gr.Button("Kelime Grafiğini Göster").click(fn=etkili_kelimeler_grafigi, outputs=kelime_output)

# 🚀 Arayüzü başlat
demo.launch()


In [ ]:
print(f"Toplam yorum sayısı: {len(df)}")
print(df['Durum'].value_counts())

In [ ]:
# 🔍 Modelin test seti üzerindeki performansı
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

y_pred = model_nb.predict(X_test_vec_nb)

print("Doğruluk:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Olumsuz", "Olumlu"])
disp.plot(cmap="YlGnBu")


In [ ]:
from sklearn.metrics import roc_curve, auc

y_probs = model_nb.predict_proba(X_test_vec_nb)[:, 1]
fpr, tpr, _ = roc_curve(y_test, y_probs)
roc_auc = auc(fpr, tpr)

print(f"AUC Skoru: {roc_auc:.3f}")


In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

cv_scores = cross_val_score(model_nb, X_train_vec_nb, y_train, cv=5)
print("CV Doğruluk Skorları:", cv_scores)
print("Ortalama:", np.mean(cv_scores))
